In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Trainer (has labels)
trainer_df = pd.read_csv('datasets/cleaned_SOLIDtest6K_trainer.tsv', sep="\t")
learner_tweets_df = pd.read_csv('datasets/cleaned_SOLID9M_learner.tsv', sep="\t")

In [2]:
import numpy as np
# trainer_labels = trainer_df['label'].values
# trainer_tweets = trainer_df['tweet'].values
learner_tweets_df['labels'] = learner_tweets_df['average'].apply(lambda x: 1 if x >= 0.5 else 0) # threshold the average values

sample_size = 80000
positive_ratio = 0.50

# Select the most confident positive values
semi_tweets_pos_df = learner_tweets_df[learner_tweets_df['average'] > 0.5].sample(n=np.floor(sample_size*positive_ratio).astype(int), random_state=1)

# Select the most confident negative values
semi_tweets_neg_df = learner_tweets_df[learner_tweets_df['average'] < 0.2].sample(n=np.floor(sample_size*(1-positive_ratio)).astype(int), random_state=1)

semi_tweets_df = pd.concat([semi_tweets_pos_df, semi_tweets_neg_df])
semi_tweets_df = semi_tweets_df.sample(frac=1, random_state=42)

semi_tweets = semi_tweets_df['text'].values
semi_labels = semi_tweets_df['labels'].values


In [3]:
semi_tweets_df.head(10)

,id,text,average,std,labels
6411147,1187178672442220544,had fish tacos the other day,0.195709,0.188270,0
2134644,1161313658473000960,think they are saying it was too windy,0.183525,0.194722,0
2772179,1161016801351700480,is the cover of the uk ed from a movie it loo...,0.178485,0.175064,0
3696591,1161756492032348160,he is had a whole pre season though,0.198720,0.201089,0
5586179,1155735694284689408,take me to the top take me take me higher,0.179318,0.194545,0
3605572,1161423107808256001,as long as this guy keeps running his mouth an...,0.561974,0.139245,1
2799246,1161552623566110720,i want to have a date with,0.194520,0.185750,0
2725602,1160937939804643328,the feeling nya btob,0.175092,0.180342,0
7611207,1187283571687473152,i could eat a horse hooves and all,0.511584,0.116744,1
4207592,1156095150449463297,hey guys i have not been on this account in ...,0.196998,0.176672,0


In [4]:
import torch
from torch.utils.data import DataLoader, Dataset


class TweetDataset(Dataset):
    def __init__(self, texts, labels=None):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        if self.labels is not None:
            label = self.labels[idx]
            return {
                'text': text,
                'label': label
            }
        else:
            return {
                'text': text
            }
        
# trainer_dataset = TweetDataset(trainer_tweets, trainer_labels)
learner_dataset = TweetDataset(semi_tweets, semi_labels)

# trainer_loader = DataLoader(trainer_dataset, batch_size=12, shuffle=True)
learner_loader = DataLoader(learner_dataset, batch_size=6, shuffle=False)

In [5]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch import optim
from tqdm import tqdm

num_epochs = 1

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


model = BertForSequenceClassification.from_pretrained('models/OLID_BERT_1', num_labels=2)
optimizer = optim.AdamW(model.parameters(), lr=1e-5)
criterion = torch.nn.CrossEntropyLoss()

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)


/home/cymn/miniconda3/envs/nvidia-newest/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [6]:
model.train()
for epoch in range(num_epochs):
    # trainer_dataloader_iterator = iter(trainer_loader)
    semi_labeled_dataloader_iterator = iter(learner_loader)
    num_batches = len(learner_loader)
    # num_batches = 5000
    for _ in tqdm(range(num_batches)):
        # Train on labeled data
        batch = next(semi_labeled_dataloader_iterator, None)
        if batch is not None:
            inputs = tokenizer(batch['text'], padding=True, truncation=True, return_tensors="pt").to(device)
            labels = torch.tensor(batch['label']).clone().detach().to(device)
            outputs = model(**inputs)
            logits = outputs.logits
            loss = criterion(logits, labels)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

  0%|          | 0/13334 [00:00<?, ?it/s]/tmp/ipykernel_5359/1995336368.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(batch['label']).clone().detach().to(device)
100%|██████████| 13334/13334 [11:24<00:00, 19.47it/s]


In [7]:
model.save_pretrained('models/SOLID_BERT_colearning_2')